In [21]:
import random
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets, metrics
from sklearn.metrics import accuracy_score

In [22]:
# reading the dataset
df = pd.read_csv("data.csv")

check = np.random.rand(len(df)) < 0.8
train = df[check]
test = df[~check]

In [23]:
# choosing 10000 samples only
N1 = train.sample(10000)

train1 = train.sample(9000)
train1 = train1.replace('A', '1')
train1 = train1.replace(['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], '-1')

X = train1.drop(['class'], axis = 1).values
y = train1['class']

In [24]:
# initial model
svmc = svm.SVC(kernel = 'linear', C = 1).fit(X, y)
f0 = svmc
svmc.predict(X[0].reshape(1, -1))

array(['-1'], dtype=object)

In [25]:
# loss function as in paper
def loss_fun(f, z):
	result = 1 - int(float(f.predict(z.drop(['class'], axis = 1).values[0].reshape(1, -1))[0])) * int(z['class'].values)
	return result

In [26]:
# m=3n
n2 = 30
k_val = 2
fi = svmc
smpl = []
flag = 0
fcurr = fi
q = 1.2
k = 0

ztplus1 = train1.sample()
for i in range(k_val + 1):
  if (i != 0):
    tobeusedlis = []
    for j in range(len(smpl)):
      tobeusedlis.append(int(smpl[j][0]))

    tempdataset = df.iloc[tobeusedlis]
    tempdataset = tempdataset.replace('A', '1')
    tempdataset = tempdataset.replace(['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], '-1')
    xtemp = tempdataset.drop(['class'], axis = 1).values
    ytemp = tempdataset['class']
    fcurr = svm.SVC(kernel = 'linear', C = 1).fit(xtemp, ytemp)

  nplus = 0
  nminus = 0
  smpl.clear()
  t = 1
  checkflag = 0
  zt = train1.sample()
  if (int(zt['class'].values) == 1):
    nplus = nplus + 1
  else:
    nminus = nminus + 1

  while (nplus + nminus < len(train1)):
    z_star = train1.sample()
    if (checkflag == 0):
      checkflag = 1
    
    if (flag == 1):
      zt = ztplus1
    
    p1 = math.exp(-loss_fun(fcurr, zt))
    p2 = math.exp(-loss_fun(fcurr, z_star))
    alpha = p2 / p1
    yt = int(zt['class'].values)
    y_star = int(z_star['class'].values)

    if (k == n2):
      flag = 1
      alpha2 = q * alpha
    
      if (alpha2 >= 1):
        smpl.append(z_star.index)
      else:
        ran_p = random.uniform(0, 1)
        if (alpha2 > ran_p):
          smpl.append(zstar.index)
          ztplus1 = z_star
    
          if (y_star == 1):
            nplus = nplus + 1
          else:
            nminus = nminus + 1
    
        else:
          k = 0
        continue

    if (alpha >= 1 and yt * y_star == 1):
      ind = z_star.index
      x = z_star.drop(['class'], axis = 1).values[0].reshape(1, -1)
      x1 = zt.drop(['class'], axis = 1).values[0].reshape(1, -1)
      P_dash = math.exp(-y_star) * int(fcurr.predict(x)[0]) / (-yt * int(fcurr.predict(x1)[0]))
      ran_p = random.uniform(0, 1)
      flag = 0
    
      if (P_dash > ran_p):
        smpl.append(z_star.index)
        if (y_star == 1):
          nplus = nplus + 1
        else:
          nminus = nminus + 1
    
        k = k + 1
      continue

    if ((yt * y_star == -1 and alpha == 1) or (alpha < 1)):
      ind = z_star.index
      x = z_star.drop(['class'], axis = 1).values[0].reshape(1, -1)
      x1 = zt.drop(['class'], axis = 1).values[0].reshape(1, -1)
      ran_p = random.uniform(0, 1)
      flag = 0
     
      if (alpha > ran_p):
        smpl.append(z_star.index)
        if (y_star == 1):
          nplus = nplus + 1
        else:
          nminus = nminus + 1
     
        k = k + 1
      continue

In [27]:
# finding the further to be processed samples
further = []
for i in range(len(smpl)):
  further.append(int(smpl[i][0]))

In [28]:
final_data = df.iloc[further]
print(final_data.head())

      class  f1  f2  f3  f4  f5  f6  f7  ...  f9  f10  f11  f12  f13  f14  f15  f16
10310     M   8   8  11   7  12   8   8  ...   4    7    6    7   12    7    8    3
7426      A   3   7   5   5   3  12   3  ...   2    9    2    9    2    5    2    8
17285     Z   7  14   7   8   5   6   7  ...   9   12    7    8    4    6    7    6
1332      X   7  10   7   5   4   6   9  ...   7   11   10    8    4   14    4    6
14270     X   4   9   6   6   4   7   6  ...   8    5    6    9    3    8    7    8

[5 rows x 17 columns]


In [29]:
# prediction using SVMC
X1 = final_data.drop(['class'], axis = 1).values
y1 = final_data['class']

new_svmc = svm.SVC(kernel = 'linear', C = 1).fit(X1, y1)

X = df.drop(['class'], axis = 1).values
y = df['class']

y_pred = new_svmc.predict(X)

print("Accuracy of the model: ", accuracy_score(y, y_pred))

Accuracy of the model:  0.848
